In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
df_attacks = pd.read_csv("DATA/attacks.csv",encoding = "ISO-8859-1")
import re
import sys
sys.path.append('../')

import src.cleaning_functions as cf

### MIS 3 HIPÓTESIS SON:
- En 2018 hubo más ataques de tiburones tigre en el hemisferio sur que en 1990.
- En México hay más ataques de tiburones blancos que en USA.
- La mayoría de las víctimas que bucean son mujeres.

Por tanto, para verificar o refutar las 3 hipótesis planteadas, voy a utilizar las siguientes columnas:
- Year
- Country
- Activity
- Sex
- Species

La forma de mi df es:

In [2]:
df_attacks.shape #Ha cambiado porque ya he quitado todas las filas de Nan.

(25723, 24)

### 1. Borro las filas que sean enteras NaN. 

In [3]:
df_attacks.dropna(how="all", inplace=True)

Voy a proceder a eliminar las columnas del dataframe que, de primeras, no me van a ser de utilidad:

In [4]:
df_attacks.drop(["Case Number","pdf","href formula","href","Case Number.1","Case Number.2","original order","Unnamed: 22","Unnamed: 23"], axis=1, inplace=True)

### 2. Borro las filas duplicadas.

Comprobamos que hemos pasado de 8703 a 6302 filas

In [5]:
borrado=df_attacks.drop_duplicates()
borrado.shape


(6302, 15)

### 3. Cambio el nombre de las columnas porque hay algunas que contienen espacios. 

Imprimiendo la lista de columnas comprobamos que solo haría falta cambiar el nombre de "Sex" y "Species".

In [6]:
para_Sex = {'Sex ': 'Sex'}

In [7]:
borrado.rename(columns=para_Sex,inplace=True)

/home/lydia/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [8]:
para_Species={'Species ': 'Species'}

In [9]:
borrado.rename(columns=para_Species, inplace=True)

Con ".info" veo qué tipos de datos tengo y veo que todos son categóricos:

In [10]:
borrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6302 entries, 0 to 6302
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    6301 non-null   object 
 1   Year                    6299 non-null   float64
 2   Type                    6297 non-null   object 
 3   Country                 6251 non-null   object 
 4   Area                    5846 non-null   object 
 5   Location                5761 non-null   object 
 6   Activity                5757 non-null   object 
 7   Name                    6091 non-null   object 
 8   Sex                     5736 non-null   object 
 9   Age                     3471 non-null   object 
 10  Injury                  6273 non-null   object 
 11  Fatal (Y/N)             5762 non-null   object 
 12  Time                    2948 non-null   object 
 13  Species                 3463 non-null   object 
 14  Investigator or Source  6284 non-null   

### 4. Remplazo los "nan" por "Unkown". 

Para ello, uso un bucle que itere por cada uno de los elementos de mi lista de columnas:

In [11]:
listacolumnas=list(borrado.columns)
print(listacolumnas)

['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species', 'Investigator or Source']


In [12]:
for elemento in listacolumnas:
    borrado[elemento].fillna(("Unknown"), inplace=True)


/home/lydia/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


### 5. Voy a empezar a armonizar los elementos de cada columna para disminuir el número de valores únicos.


### 5.1. Usando ".replace" cuando no hay muchos valores únicos: "Sex", "Type" & "Fatal (Y/N)".

#### A. Sex.

Empiezo por la columna de "Sex".
- Compruebo los valores únicos de la columna "Sex".
- Sustituyo la "M con espacio" por la "M normal". El resto las replazo por "unkonwn" y me quedo únicamente por M y F. 
- Compruebo de nuevo los valores únicos de "Sex" y veo que se han reducido a:"F","M","nan" y "Unknown".

In [35]:
borrado["Sex"].unique()

df2=borrado.replace({'M ':'M','lli':'Unknown','N':'Unknown','.':'Unknown'})

df2["Sex"].unique()


array(['F', 'M', 'Unknown'], dtype=object)

#### B. Type.

Veo cuáles son los valores únicos de la columna "Type" y luego hago replace para armonizar sus elementos y reducirlos a 5:

In [14]:
#tipo=df2["Type"].unique()
df2["Type"].value_counts()
df2.replace({"Questionable":"Invalid","Boat":"Boating","Boatomg":"Boating"},inplace=True)
df2["Type"].value_counts()

Unprovoked      4594
Provoked         574
Invalid          549
Boating          341
Sea Disaster     239
Unknown            5
Name: Type, dtype: int64

#### C. Fatal (Y/N)

Ahora voy a ver los valores únicos de la columna "Fatal (Y/N)", dejando únicamente 3:Y,N y Unkown.

In [15]:
df2["Fatal (Y/N)"].value_counts()
df2["Fatal (Y/N)"].replace({"UNKNOWN":"Unknown"," N":"N","y":"Y","M":"Unknown","2017":"Unknown","N ":"Unknown","Unkown":"Unknown" },inplace=True)
df2["Fatal (Y/N)"].value_counts()

Unknown    4906
Y          1389
N             7
Name: Fatal (Y/N), dtype: int64

### 5.2. Usando Regex: Species, Activitiy, Injury.

#### A. Species.

In [16]:
especies=df2["Species"].unique()

In [17]:
df2["Species"]= df2["Species"].apply(cf.funcion_valores_unicos)

In [18]:
df2.Species.value_counts()

Others                       4093
White shark                   667
Involvement not confirmed     327
Tiger shark                   283
Bull shark                    175
Blacktip shark                101
Nurse shark                    97
Whaler shark                   70
Blue shark                     56
Wobbegong shark                50
Hammerhead shark               48
Mako shark                     44
Raggedtooth shark              42
Lemon shark                    37
Zambesi shark                  29
Grey shark                     25
Sand shark                     24
Red shark                      23
Spinner shark                  21
Caribbean reef shark           16
Sevengill shark                13
Dusky shark                    11
Porbeagle shark                 7
Galapagos shark                 7
Angel shark                     6
Copper shark                    5
Silky shark                     4
Thresher shark                  4
Salmon shark                    3
Dogfish shark 

#### B. Activity.

In [19]:
activities=df2["Activity"].unique()

In [20]:
df2["Activity"]= df2["Activity"].apply(cf.valores_unicos_activity)

In [21]:
df2.Activity.value_counts()

Others                  2489
Fishing                 1171
Swimming                1106
Diving                   511
Bathing                  185
Interact with sharks     154
Standing in water        141
Body Boarding            136
Boat                     105
OverBoard                 85
Boogie Board              51
Floating                  45
Paddling                  37
Walking                   26
Wind Surfing              23
Kite                      22
Jumping                   15
Name: Activity, dtype: int64

#### C. Injury.

In [36]:
injury=df2["Injury"].unique()

In [37]:
df2["Injury"] = df2["Injury"].apply(cf.valores_unicos_injury)

In [38]:
df2.Injury.value_counts()

Fatal              1183
Others             1135
Leg injury          889
No injury           804
Laceration          555
Foot Injury         475
Hand injury         461
Thigh Injury        194
Minor Injury        159
Calf Injury         136
Ankle Injury         84
Shoulder Injury      50
Knee Injury          40
Torso Injury         34
Abrasion Injury      26
Elbow Injury         24
Chest Injury         23
Feet Injury          17
Puncture wounds      13
Name: Injury, dtype: int64

In [39]:
list(df2.Injury.unique())

['No injury',
 'Minor Injury',
 'Leg injury',
 'Hand injury',
 'Fatal',
 'Others',
 'Laceration',
 'Ankle Injury',
 'Foot Injury',
 'Feet Injury',
 'Shoulder Injury',
 'Calf Injury',
 'Abrasion Injury',
 'Torso Injury',
 'Thigh Injury',
 'Elbow Injury',
 'Knee Injury',
 'Puncture wounds',
 'Chest Injury']

### 5.3. Otros: Country, Name, Year.

#### a. Country.

Obtengo los valores únicos de la columa Países:

In [22]:
(df2["Country"].unique())

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA',
       'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA',
       'ECUADOR', 'MALAYSIA', 'LIBYA', 'Unknown', 'CUBA', 'MAURITIUS',
       'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN',
       'EGYPT', 'ST HELENA, British overseas territory', 'COMOROS',
       'REUNION', 'FRENCH POLYNESIA', 'UNITED KINGDOM',
       'UNITED ARAB EMIRATES', 'PHILIPPINES', 'INDONESIA', 'CHINA',
       'COLUMBIA', 'CAPE VERDE', 'Fiji', 'DOMINICAN REPUBLIC',
       'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE', 'FIJI', 'PUERTO RICO',
       'ITALY', 'ATLANTIC OCEAN', 'GREECE', 'ST. MARTIN', 'FRANCE',
       'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL',
       'DIEGO GARCIA', 'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES',
       'GUAM', 'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND',
       'CANADA', 'CROATIA', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA',
       'RUSSIA', 'TURKS & CAICOS', 

Ahora usando ".apply" importo y aplico la función que he creado en "Cleaning_functions.py"para determinar si cada país de la columna Country pertenece al Hemisferio Norte o al Hemisferio Sur y veo que efectivamente se crea otra columna llamada Hemisferio.

In [23]:
df2["Hemisphere"] = df2["Country"].apply(cf.hemisferio) 

#### b. Name.

Tras comprobar los valores únicos de esta columna y, al no ser de utilidad, para verificar o refutar mis tres hipótesis, dejo la columna "Name" tal y como estaba.

In [40]:
nombres=list(df2["Name"].unique())
frecuencianombres=list(df2["Name"].value_counts(sort=False)) #Podemos incluir el parámetro "Sort=False" para que no me ordene la frecuenecia con la que aparecen los valores únicos de forma descendente y lo deje en el miso orden que aparecen en el dataframe.

#### F. Year.

A través de una función que he creado e importado de mi archivo de funciones de Visual Code, limpio los valores de esta columna que eran flotantes y los convierto en números enteros.

In [32]:
años=df2["Year"].unique()

In [33]:
df2["Year"]=df2["Year"].apply(cf.años)

### Importo el csv limpio:

In [34]:
df2.to_csv('Data/attacks_clean.csv')